In [8]:
import cv2 
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import cifar10 # importing the dataset
from keras import backend as K 
from keras.utils import np_utils
from keras.layers import Layer
import keras.backend as K
import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D, AveragePooling2D,Flatten

In [12]:
num_classes = 10
def cifar10_data(img_rows, img_cols):
# Load training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()
# Resize images to 244x244
    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:,:,:,:][:3000]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in   X_valid[:,:,:,:][:3000]])
    Y_train = Y_train[:3000]
    Y_valid = Y_valid[:3000]
# Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train, num_classes)
    Y_valid = np_utils.to_categorical(Y_valid, num_classes)
 
    X_train = X_train.astype('float32')
    X_valid = X_valid.astype('float32')
# making all the values range between 0 and 1
    X_train = X_train / 255.0
    X_valid = X_valid / 255.0
    return X_train, Y_train, X_valid, Y_valid

In [13]:
X_train, y_train, X_test, y_test = cifar10_data(224, 224)

In [22]:
from tensorflow.keras.initializers import GlorotUniform

# Use a fixed seed for reproducibility
glorot_initializer = GlorotUniform(seed=1)

# Then use this initializer in your model layers
layer = keras.layers.Dense(64, kernel_initializer=glorot_initializer)


In [23]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,
    filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):
 
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
 
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)
    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)
    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)
    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    return output

kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [24]:
input_layer = Input(shape=(224, 224, 3))
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)
x = inception_module(x,filters_1x1=64,filters_3x3_reduce=96,filters_3x3=128,filters_5x5_reduce=16,filters_5x5=32,filters_pool_proj=32,name='inception_3a')
x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,filters_3x3=192,filters_5x5_reduce=32,filters_5x5=96,filters_pool_proj=64,name='inception_3b')
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)
x = inception_module(x,filters_1x1=192,filters_3x3_reduce=96,filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64,name='inception_4a')
x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)
x = inception_module(x,filters_1x1=160,filters_3x3_reduce=112,filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4b')
x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4c')
x = inception_module(x,filters_1x1=112,filters_3x3_reduce=144,filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64,filters_pool_proj=64,name='inception_4d')
x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)
x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_4e')
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)
x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_5a')
x = inception_module(x, filters_1x1=384,filters_3x3_reduce=192,filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128,filters_pool_proj=128,name='inception_5b')
x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)
x = Dropout(0.4)(x)
x = Dense(10, activation='softmax', name='output')(x)
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [26]:
model.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 56, 56, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                     

                                                                                                  
 conv2d_134 (Conv2D)            (None, 14, 14, 24)   12312       ['inception_4a[0][0]']           
                                                                                                  
 max_pooling2d_21 (MaxPooling2D  (None, 14, 14, 512)  0          ['inception_4a[0][0]']           
 )                                                                                                
                                                                                                  
 conv2d_131 (Conv2D)            (None, 14, 14, 160)  82080       ['inception_4a[0][0]']           
                                                                                                  
 conv2d_133 (Conv2D)            (None, 14, 14, 224)  226016      ['conv2d_132[0][0]']             
                                                                                                  
 conv2d_13

                                                                                                  
 conv2d_159 (Conv2D)            (None, 7, 7, 32)     26656       ['max_pool_4_3x3/2[0][0]']       
                                                                                                  
 max_pooling2d_25 (MaxPooling2D  (None, 7, 7, 832)   0           ['max_pool_4_3x3/2[0][0]']       
 )                                                                                                
                                                                                                  
 conv2d_156 (Conv2D)            (None, 7, 7, 256)    213248      ['max_pool_4_3x3/2[0][0]']       
                                                                                                  
 conv2d_158 (Conv2D)            (None, 7, 7, 320)    461120      ['conv2d_157[0][0]']             
                                                                                                  
 conv2d_16

In [30]:
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import math

epochs = 15
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

# Update optimizer instantiation with `learning_rate` instead of `lr`
sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)

# Create a LearningRateScheduler callback
lr_sc = LearningRateScheduler(decay, verbose=1)

# Compile the model with the updated optimizer
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [31]:
H = model.fit(X_train, [y_train, y_train, y_train],validation_data=(X_test, [y_test, y_test, y_test]), epochs=epochs, batch_size=256, callbacks=[lr_sc])


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/15


ResourceExhaustedError: Graph execution error:

Detected at node 'inception_v1/conv_2b_3x3/1/Relu' defined at (most recent call last):
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\jayat\anaconda3\envs\tf_v1\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\jayat\anaconda3\en